In [8]:
import polars as pl
import intervaltree

In [9]:
df_freq = pl.read_csv("HG00731_binned_freq.bed", separator="\t", has_header=False, new_columns=["chrom", "st", "end", "cov", "avg_methyl"])
df_freq

chrom,st,end,cov,avg_methyl
str,i64,i64,f64,f64
"""haplotype1-000…",0,5000,72.58,19.77
"""haplotype1-000…",5000,10000,68.04,18.06
"""haplotype1-000…",10000,15000,81.43,19.76
"""haplotype1-000…",15000,20000,72.99,19.24
"""haplotype1-000…",20000,25000,81.96,17.9
…,…,…,…,…
"""haplotype2-000…",3905000,3910000,60.26,23.62
"""haplotype2-000…",3910000,3915000,56.71,22.72
"""haplotype2-000…",3915000,3920000,45.48,21.09


In [10]:
df_freq_normalized = df_freq.with_columns(
    mean_avg_methyl_chrom=pl.col("avg_methyl").mean().over("chrom"),
    std_avg_methyl_chrom=pl.col("avg_methyl").std().over("chrom"),
)
df_freq_normalized

chrom,st,end,cov,avg_methyl,mean_avg_methyl_chrom,std_avg_methyl_chrom
str,i64,i64,f64,f64,f64,f64
"""haplotype1-000…",0,5000,72.58,19.77,25.647138,4.150232
"""haplotype1-000…",5000,10000,68.04,18.06,25.647138,4.150232
"""haplotype1-000…",10000,15000,81.43,19.76,25.647138,4.150232
"""haplotype1-000…",15000,20000,72.99,19.24,25.647138,4.150232
"""haplotype1-000…",20000,25000,81.96,17.9,25.647138,4.150232
…,…,…,…,…,…,…
"""haplotype2-000…",3905000,3910000,60.26,23.62,22.555369,inf
"""haplotype2-000…",3910000,3915000,56.71,22.72,22.555369,inf
"""haplotype2-000…",3915000,3920000,45.48,21.09,22.555369,inf
